<a href="https://colab.research.google.com/github/durvibangera/nlp/blob/main/nlp_12_kindle_reviews_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


1. Preprocessing and Cleaning
2. Train Test Split
3. BOW, TFIDF, Word2Vec
3. Train ML algorithms

In [2]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/all_kindle_review.csv')

In [3]:
df.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


for this use case we will only need the reviewText column and the rating column since we're doing nlp

In [4]:
df = df[['reviewText','rating']]
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [5]:
df.shape

(12000, 2)

In [6]:
df.isnull().sum()

,0
reviewText,0
rating,0


In [7]:
df['rating'].unique()

array([3, 5, 4, 2, 1])

In [8]:
df['rating'].value_counts()

,count
rating,
5,3000
4,3000
3,2000
2,2000
1,2000


## preprocessing and cleaning

In [9]:
# postive review is 1 and negative review is 0
df['rating'] = df['rating'].apply(lambda x:0 if x<3 else 1)

In [10]:
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",1
1,Great short read. I didn't want to put it dow...,1
2,I'll start by saying this is the first of four...,1
3,Aggie is Angela Lansbury who carries pocketboo...,1
4,I did not expect this type of book to be in li...,1


In [11]:
df['rating'].unique()

array([1, 0])

In [12]:
df['rating'].value_counts()

,count
rating,
1,8000
0,4000


In [13]:
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",1
1,Great short read. I didn't want to put it dow...,1
2,I'll start by saying this is the first of four...,1
3,Aggie is Angela Lansbury who carries pocketboo...,1
4,I did not expect this type of book to be in li...,1


In [14]:
## lower all the cases
df['reviewText'] = df['reviewText'].str.lower()

In [15]:
df.head()

,reviewText,rating
0,"jace rankin may be short, but he's nothing to ...",1
1,great short read. i didn't want to put it dow...,1
2,i'll start by saying this is the first of four...,1
3,aggie is angela lansbury who carries pocketboo...,1
4,i did not expect this type of book to be in li...,1


In [16]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [17]:
from bs4 import BeautifulSoup

In [18]:
## Removing special characters
df['reviewText']=df['reviewText'].apply(lambda x:re.sub('[^a-z A-z 0-9-]+', '',x))
## Remove the stopswords
df['reviewText']=df['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))
## Remove url
df['reviewText']=df['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))
## Remove html tags
df['reviewText']=df['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())
## Remove any additional spaces
df['reviewText']=df['reviewText'].apply(lambda x: " ".join(x.split()))

In [19]:
df.head()

,reviewText,rating
0,jace rankin may short hes nothing mess man hau...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four books wasnt expect...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,expect type book library pleased find price right,1


In [20]:
lemmatizer = WordNetLemmatizer()

In [21]:
def lemmatize_words(text):
  return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [22]:
df['reviewText']=df['reviewText'].apply(lambda x:lemmatize_words(x))

In [23]:
df.head()

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four book wasnt expecti...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1


## trtain test split

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['reviewText'], df['rating'], test_size=0.2, random_state=42)

## bow

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer()
X_train_bow = bow.fit_transform(X_train).toarray()
X_test_bow = bow.transform(X_test).toarray()

## tfidf

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train).toarray()
X_test_tfidf = tfidf.transform(X_test).toarray()

## appplying model

In [27]:
from sklearn.naive_bayes import GaussianNB
nb_model_bow = GaussianNB().fit(X_train_bow, y_train)
nb_model_tfidf = GaussianNB().fit(X_train_tfidf, y_train)

In [28]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [29]:
y_pred_bow = nb_model_bow.predict(X_test_bow)
y_pred_tfidf = nb_model_tfidf.predict(X_test_tfidf)

In [30]:
print("Bow: ",accuracy_score(y_test, y_pred_bow))
print("TFIDF: ",accuracy_score(y_test, y_pred_tfidf))

Bow:  0.5745833333333333
TFIDF:  0.57875


In [31]:
confusion_matrix(y_test, y_pred_bow)

array([[499, 304],
       [717, 880]])

In [32]:
confusion_matrix(y_test, y_pred_tfidf)

array([[488, 315],
       [696, 901]])

accuracy not good so better to do word2vec

In [33]:
!pip install gensim

In [34]:
import gensim
from gensim.models import Word2Vec, KeyedVectors

In [35]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [37]:
from gensim.models import Word2Vec
# Train Word2Vec model
word2vec_model = Word2Vec(sentences=[doc.split() for doc in X_train], vector_size=100, window=5, min_count=1, workers=4)

# Function to generate document vectors by averaging word vectors
def document_vector(doc):
    doc = [word for word in doc.split() if word in word2vec_model.wv.index_to_key]
    if not doc:
        return None
    return sum(word2vec_model.wv[word] for word in doc) / len(doc)

# Generate Word2Vec vectors for train and test sets
X_train_w2v = X_train.apply(document_vector)
X_test_w2v = X_test.apply(document_vector)

# Handle potential None values in the generated vectors
X_train_w2v = X_train_w2v.dropna()
X_test_w2v = X_test_w2v.dropna()

# Align the labels with the generated vectors after dropping None values
y_train_w2v = y_train[X_train_w2v.index]
y_test_w2v = y_test[X_test_w2v.index]

print("Shape of X_train_w2v:", (len(X_train_w2v), len(X_train_w2v.iloc[0]) if len(X_train_w2v) > 0 else 0))
print("Shape of X_test_w2v:", (len(X_test_w2v), len(X_test_w2v.iloc[0]) if len(X_test_w2v) > 0 else 0))
print("Shape of y_train_w2v:", y_train_w2v.shape)
print("Shape of y_test_w2v:", y_test_w2v.shape)

Shape of X_train_w2v: (9600, 100)
Shape of X_test_w2v: (2400, 100)
Shape of y_train_w2v: (9600,)
Shape of y_test_w2v: (2400,)


In [38]:
nb_model_w2v = GaussianNB().fit(list(X_train_w2v), y_train_w2v)
y_pred_w2v = nb_model_w2v.predict(list(X_test_w2v))

In [39]:
accuracy_score(y_test_w2v, y_pred_w2v)

0.6945833333333333

In [40]:
print(classification_report(y_test_w2v, y_pred_w2v))

              precision    recall  f1-score   support

           0       0.53      0.78      0.63       803
           1       0.85      0.65      0.74      1597

    accuracy                           0.69      2400
   macro avg       0.69      0.72      0.68      2400
weighted avg       0.75      0.69      0.70      2400



In [43]:
print(confusion_matrix(y_test_w2v, y_pred_w2v))

[[ 624  179]
 [ 554 1043]]
